In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy.interpolate import interp1d

In [3]:
dir = '/gdrive/My Drive/YL TEZ/py/data_whoi/Bottom Profile/XLSX'

In [4]:
os.chdir(dir)

In [5]:
file_list = glob.glob('*.xlsx')

In [6]:
len(file_list)

39

In [ ]:
file_list

In [18]:
Data = pd.read_excel(file_list[22], index_col=0)
Data

,FRF,Profile Number,Survey Number,Latitude,Longitude,Northing,Easting,xFRF,yFRF,Elevation,Elipsoid,Date,Time,Time(h)
0,FRF,709,723,36.183798,-75.751598,901757.227,274775.176,50.850,707.374,7.040,0,19941011,0,0
1,FRF,709,723,36.183806,-75.751572,901759.524,274776.101,53.325,707.456,7.367,0,19941011,0,0
2,FRF,709,723,36.183817,-75.751554,901761.164,274777.396,55.309,708.111,7.263,0,19941011,0,0
3,FRF,709,723,36.183817,-75.751544,901762.035,274777.425,56.138,707.840,7.172,0,19941011,0,0
4,FRF,709,723,36.183819,-75.751541,901762.338,274777.662,56.504,707.959,7.175,0,19941011,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6335,FRF,1234,723,36.189479,-75.748882,901980.764,275413.232,479.365,1230.307,-4.912,0,19941011,133800,49080
6336,FRF,1234,723,36.189482,-75.748876,901981.305,275413.588,479.996,1230.456,-4.916,0,19941011,133800,49080
6337,FRF,1234,723,36.189485,-75.748869,901981.959,275413.975,480.743,1230.596,-4.925,0,19941011,133800,49080
6338,FRF,1234,723,36.189489,-75.748860,901982.745,275414.353,481.611,1230.682,-4.937,0,19941011,133800,49080


In [ ]:
def lowest(a, n): return np.partition(a, n-1)[:n]
in_list = Data.iloc[:, 1]

## Find small value

In [ ]:
given_value = 929

In [ ]:
for k in range(len(in_list)):
  if in_list[k] < given_value:
    small_value = in_list[k]
  else:
    break
print(small_value)

In [ ]:
Sm = Data.loc[Data['Profile Number'] == small_value]
Sm = Sm[['Profile Number', 'xFRF', 'Elevation']]
Sm

## Find higher value

In [ ]:
for t in range(len(in_list)):
  if in_list[t] > given_value:
    high_value = in_list[t]
    break
  else:
    continue
print(high_value)

In [ ]:
Hi = Data.loc[Data['Profile Number'] == high_value]
Hi = Hi[['Profile Number', 'xFRF', 'Elevation']]
Hi

In [ ]:
x_values = Hi[['xFRF']].values.tolist()
minx = int(min(x_values)[0])
maxx = int(max(x_values)[0])

## Interpolation x values with Elevations

In [ ]:
x = np.arange(minx+1, maxx, 1)

x_veri = Hi.xFRF.values
d_veri = Hi.Elevation.values


f = interpolate.interp1d(x_veri, d_veri, kind='linear')
d = f(x)

In [ ]:
data1 = {'x': x, 'Elevation':d}
Hi_val = pd.DataFrame(data=data1)
Hi_val

In [ ]:
x_values = Sm[['xFRF']].values.tolist()
minx = int(min(x_values)[0])
maxx = int(max(x_values)[0])

In [ ]:
x = np.arange(minx+1, maxx, 1)

x_veri = Sm.xFRF.values
d_veri = Sm.Elevation.values


f = interpolate.interp1d(x_veri, d_veri, kind='linear')
d = f(x)

In [ ]:
data2 = {'x': x, 'Elevation':d}
Sm_val = pd.DataFrame(data=data2)
Sm_val

## Tail values of y=929

In [ ]:
if max(Sm_val[['x']].values.tolist()) > max(Hi_val[['x']].values.tolist()):
  #Tail = Sm_val.loc[['x'] >= ]
  Tail =  Sm_val.loc[Sm_val['x'] > max(Hi_val[['x']].values.tolist())[0]]
  Inter = Sm_val.loc[Sm_val['x'] < max(Hi_val[['x']].values.tolist())[0]]
else:
  Tail = Hi_val.loc[Hi_val['x'] > max(Sm_val[['x']].values.tolist())[0]]
  Inter = Sm_val.loc[Sm_val['x'] < max(Hi_val[['x']].values.tolist())[0]]

## Intermediate Values

In [ ]:
Inter[['Elevation']]

In [ ]:
new_elevations = list()
for t in range(len(Inter.x.values)):
  x1 = Inter.iloc[t-1,0]
  x2 = Inter.iloc[t,0]
  d1 = Inter.iloc[t-1, 1]
  d2 = Inter.iloc[t, 1]
  new_elevations.append(d1+((d2-d1)/(x2-x1)))
Inter = Inter.drop(['Elevation'], axis=1)
#new_elevations
Inter['Elevation'] = new_elevations
Inter = Inter.drop(0)
Inter.reset_index(drop=True, inplace=True)

In [ ]:
Data_Final = pd.concat([Inter, Tail])

In [ ]:
date = str(Data.iloc[0, 11])

In [ ]:
os.chdir(dir + '/929 Profiles')

In [ ]:
Data_Final.to_excel(date + ".xlsx", sheet_name="y929_" + date)

In [ ]:
%ls